In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen as uReq
import numpy as np
import pandas as pd
from time import sleep
import requests
import re

In [2]:
mydf = pd.read_csv('GFM_url_list.csv', sep = '\t')

In [3]:
headers = ["Url", "Category","Position", "Title", "Location","Amount_Raised", "Goal", "Number_of_Donators", "Length_of_Fundraising", "FB_Shares", "GFM_hearts", "Text"]
mydf = mydf.reindex(columns = headers)

full_df = pd.DataFrame(columns = headers)
#need to scrape a single url now

In [4]:
def scrape_url(row_index):
    single_row = mydf.iloc[row_index]
    url = single_row["Url"]
    category = single_row["Category"]
    position = single_row["Position"]
    
    page = requests.get(url)
         
    soup = BeautifulSoup(page.text, 'lxml')

    try:
        content_dic = json.loads(re.sub(';$','',soup.find('script').text.split('= ')[1]))

        amount_raised = content_dic['feed']['campaign']['current_amount']
        
        goal = content_dic['feed']['campaign']['goal_amount']
        
        NumDonators = content_dic['feed']['campaign']['donation_count']
        
        timeFundraised = (pd.Timestamp.now() - (pd.to_datetime(content_dic['feed']['campaign']['created_at'])).astimezone(tz=None))
    except:
        amount_raised = np.nan
        goal = np.nan
        NumDonators = np.nan
        timeFundraised = np.nan
        
    
    title_container = soup.find_all("h1",{"class":"a-campaign-title"})#<h1 class="campaign-title">Help Rick Muchow Beat Cancer</h1>
    
    try:
        title = title_container[0].text
    except:
        title = np.nan
    
    text_container =  soup.find('meta', attrs={'name': 'description'})
    
    try:
        all_text = text_container['content']
    except:
        all_text = np.nan
    
    try:
        FB_shares_container = soup.find_all("strong", {"class":"js-share-count-text"})
        FB_shares = FB_shares_container[0].text.splitlines()
        FB_shares = FB_shares[1].replace(" ", "").replace("\xa0", "")
    except:
        FB_shares = np.nan
        
    try:
        hearts_container = soup.find_all("div", {"class":"campaign-sp campaign-sp--heart fave-num"})
        hearts = hearts_container[0].text
    except:
        hearts = np.nan
    
    try:
        content_dic = json.loads(re.sub(';$','',soup.find('script').text.split('= ')[1]))
        location = content_dic['feed']['campaign']['location']['city']
    except:
        location = np.nan
        
    temp_row = np.array([[url, category, position, title, location, amount_raised, goal, NumDonators, timeFundraised, FB_shares, hearts, all_text]])
    temp_df = pd.DataFrame(temp_row, columns = headers)
    
    return(temp_df)

In [5]:
def scrape_all_urls(file = 'new.csv', start = 0, end = len(mydf)):
    for i in range(start, end):
        try:
            temp_df = scrape_url(i)
            temp_df.to_csv(file, mode = 'a',sep = '\t', header = False)
            print("Scraping url %s" %(i+1))
        except:
            sleep(5)
            temp_df = scrape_url(i)
            temp_df.to_csv(file, mode = 'a',sep = '\t', header = False)
            print("Scraping url %s" %(i+1))

In [ ]:
scrape_all_urls()

Scraping url 1
Scraping url 2
Scraping url 3
Scraping url 4
Scraping url 5
Scraping url 6
Scraping url 7
Scraping url 8
Scraping url 9
Scraping url 10
Scraping url 11
Scraping url 12
Scraping url 13
Scraping url 14
Scraping url 15
Scraping url 16
Scraping url 17
Scraping url 18
Scraping url 19
Scraping url 20
Scraping url 21
Scraping url 22
Scraping url 23
Scraping url 24
Scraping url 25
Scraping url 26
Scraping url 27


In [7]:
scrape_url(0)

,Url,Category,Position,Title,Location,Amount_Raised,Goal,Number_of_Donators,Length_of_Fundraising,FB_Shares,GFM_hearts,Text
0,https://www.gofundme.com/f/justiceforjacobblake,Medical,0,Justice for Jacob Blake,nan,nan,nan,nan,nan,nan,nan,On August 23rd my son was shot multiple times ...


In [35]:
url = mydf.iloc[0][0]
client = uReq(url)  # grabs the page
soup = BeautifulSoup(client.read(), 'html.parser')  # using the default html parser


#contains amount raised - goal amount - # of donators - length of fundraising
    
#container = soup.find_all("span",{"class":"text-stat-value"})
container = soup.find("span", {"class":"text-stat-value"})
#info_string = container[0].text
#info_string = info_string.splitlines()
#info_string = list(filter(None, info_string))
        
#amount_raised = int(info_string[0][1:].replace(',',''))
        
#goal = re.findall('\$(.*?) goal', info_string[1])[0]
        
#NumDonators = re.findall('by (.*?) people', info_string[2])[0]
soup

<!DOCTYPE html>
<html data-reactroot="" lang="en"><head><title data-react-helmet="true">Fundraiser by Julia Jackson : Justice for Jacob Blake</title><meta charset="utf-8" data-react-helmet="true"/><meta content="ie=edge" data-react-helmet="true" http-equiv="x-ua-compatible"/><meta content="width=device-width, initial-scale=1.0" data-react-helmet="true" name="viewport"/><meta content="On August 23rd my son was shot multiple times in the back by a Kenosha Police Department officer… Julia Jackson needs your support for Justice for Jacob Blake" data-react-helmet="true" name="description"/><meta content="407682420960" data-react-helmet="true" property="fb:app_id"/><meta content="299947508549" data-react-helmet="true" property="fb:pages"/><meta content="455380344852029" data-react-helmet="true" property="fb:pages"/><meta content="On August 23rd my son was shot multiple times in the back by a Kenosha Police Department officer… Julia Jackson needs your support for Justice for Jacob Blake" data

In [34]:
mydf.iloc[0][0]

'https://www.gofundme.com/f/justiceforjacobblake'